# Controlling Subordinate Devices with JSON Commands
This product is developed using a brain-like architecture, where the main control unit communicates with the subordinate device via serial ports (Raspberry Pi through GPIO serial ports). It's important to note that this chapter serves as a precursor to a more detailed exploration of the JSON command set for the subordinate device, and its content is similar to the previous chapter on Python chassis movement control. If you're already familiar with that chapter, you might find this overview sufficient.

## Advantages of JSON data format

JSON (JavaScript Object Notation) is a lightweight data interchange format that has become one of the standards for data transmission on the internet. Here are some advantages of JSON
:- High Readability: JSON uses a text format that is easy for humans to read and write. It organizes data in key-value pairs, making it more readable and understandable during transmission and storage.。
- Lightweight: JSON syntax is more concise and compact compared to other data formats like XML, making it more lightweight. This reduces the size of data transmissions and network bandwidth usage, improving transmission efficiency.。
- Ease of Parsing: The simple and clear data structure of JSON makes it easy to parse and serialize. Nearly all programming languages offer libraries for parsing and generating JSON, allowing developers to easily work with JSON data.。
- Wide Language Compatibility: JSON is supported in almost all programming languages, enabling convenient data exchange and communication across different platforms and systems.。
- Support for Multiple Data Types: JSON supports a variety of data types, including strings, numbers, boolean values, arrays, and objects. This flexibility allows it to represent a wide range of data structures.。
- Seamless Integration with Web Technologies: JSON originated from JavaScript, making its integration with web technologies very tight. It is highly compatible with the JavaScript language, making it convenient to use in web applications.。

## A Simple Example of Controlling a Subordinate Device with JSON Commands

In the following example, we use the is_raspberry_pi5() function to determine the current model of the Raspberry Pi because the GPIO serial port device names differ between Raspberry Pi 4B and Raspberry Pi 5. It's crucial to use the correct GPIO device name and the same baud rate as the subordinate device (default is 115200).

Before executing the code block below, ensure the robot is elevated so that the drive wheels are off the ground. Activating the code will cause the robot to move; take precautions to prevent it from falling off the table.

In [ ]:
from base_ctrl import BaseController
import time

# Function for Detecting Raspberry Pi
def is_raspberry_pi5():
    with open('/proc/cpuinfo', 'r') as file:
        for line in file:
            if 'Model' in line:
                if 'Raspberry Pi 5' in line:
                    return True
                else:
                    return False

# Determine the GPIO Serial Device Name Based on the Raspberry Pi Model
if is_raspberry_pi5():
    base = BaseController('/dev/ttyAMA0', 115200)
else:
    base = BaseController('/dev/serial0', 115200)

# The wheel rotates at a speed of 0.2 meters per second and stops after 2 seconds.
base.send_command({"T":1,"L":0.2,"R":0.2})
time.sleep(2)
base.send_command({"T":1,"L":0,"R":0})

By invoking the code block mentioned above, the Raspberry Pi will initially send the command {"T":1,"L":0.2,"R":0.2} (the structure of commands will be discussed in more detail in later chapters). This command starts the wheels turning. After a two-second interval, the Raspberry Pi will send another command {"T":1,"L":0,"R":0}, causing the wheels to stop. It's important to note that even if the command to stop the wheels isn't sent, the wheels will still cease turning if no new commands are issued. This is due to a heartbeat function built into the subordinate machine. The purpose of this heartbeat function is to halt the current motion command automatically if the master control hasn't sent any new commands to the subordinate machine for an extended period. This function is designed to prevent continuous motion of the subordinate machine in case the master control encounters a problem that leads to a freeze or crash.

If you want the robot to continue moving indefinitely, the master control unit needs to cyclically send motion control commands every 2 to 4 seconds.